# Data Preparation Notebook

### Introduction
This notebook serves as a tool used to do any additional filtering or cleaning after extracting the data from online. Any extra data processing done here will be justified and the final result will be saved in a file that can be used for later. 

    
#### Preparation steps and justifications
Each filtering action is justified by findings obtained during EDA. The list below should outline what specific type of transformation is done along with the reason why it needs to be done.

**Include only "Yes"/"No" values in "Coronary heart disease" column**
- Since the scope of this project is to predict whether or not a patient has CHD, the values in this column must be concise and clear; we cannot trust naively mapping "null" values to "no" or "dont know" to "no" values.


In [1]:
cleaned_data_filename = 'nhanes_data_processed.parquet'

In [2]:
import numpy as np
import pandas as pd
from IPython.display import Markdown, display
import os

# Display all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

data_directory = 'data'
raw_data_filename = 'nhanes_data.parquet'
resulting_filepath = os.path.join(data_directory, cleaned_data_filename)
original_filepath = os.path.join(data_directory, raw_data_filename)

### Read in our dataset

In [3]:
df = pd.read_parquet(original_filepath)
print('Dataset size:', df.shape)
df.head()

Dataset size: (101316, 38)


,SEQN,Weight,Body mass index,Year Range,Systolic,Diastolic,Gender,Age,Diabetes,Glycohemoglobin,Cholesterol,High-density lipoprotein (HDL),Albumin,Alanine aminotransferase (ALT),Aspartate aminotransferase (AST),Alkaline phosphatase (ALP),Gamma-glutamyl transferase (GGT),Glucose,Iron,Lactate dehydrogenase (LDH),Phosphorus,Bilirubin,Protein,Triglycerides,Uric acid,Creatinine,White blood cells,Basophils,Red blood cells,Hemoglobin,Red blood cell width,Platelet count,Mean volume of platelets,Coronary heart disease,Blood related diabetes,Blood related stroke,Moderate-work,Vigorous-work
0,1.0,12.5,14.90,1999-2000,NaN,NaN,2.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,75.4,24.90,1999-2000,106.0,58.0,1.0,926.0,2.0,4.7,5.56,1.39,45.0,16.0,19.0,62.0,20.0,78.0,11.28,140.0,1.066,12.0,72.0,1.298,362.8,61.9,7.6,5.397605e-79,4.73,14.1,13.7,214.0,7.7,2.0,2.0,2.0,NaN,3.0
2,3.0,32.9,17.63,1999-2000,110.0,60.0,2.0,125.0,2.0,NaN,3.34,0.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.5,5.397605e-79,4.52,13.7,11.7,270.0,8.6,NaN,NaN,NaN,NaN,NaN
3,4.0,13.3,NaN,1999-2000,NaN,NaN,1.0,22.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8,1.000000e-01,4.77,9.3,15.3,471.0,7.8,NaN,NaN,NaN,NaN,NaN
4,5.0,92.5,29.10,1999-2000,122.0,82.0,1.0,597.0,2.0,5.5,7.21,1.08,45.0,28.0,22.0,63.0,34.0,95.0,24.54,133.0,1.033,8.6,73.0,3.850,404.5,70.7,5.9,5.397605e-79,5.13,14.5,13.1,209.0,10.4,2.0,2.0,2.0,17.0,1.0


## Mapping Enumerated Columns to Categorical Values

In [4]:
codebook_df = pd.read_csv('data/codebook.csv')
print('Codebook size:', codebook_df.shape)
codebook_df.head()

Codebook size: (575, 8)


,year_range,attribute,attribute_name,description,is_range,start_range_value,end_range_value,single_value
0,1999-2000,RIAGENDR,Gender,Male,False,NaN,NaN,1.0
1,1999-2000,RIAGENDR,Gender,Female,False,NaN,NaN,2.0
2,1999-2000,RIDAGEMN,Age,Range of Values,True,0.0,1019.0,NaN
3,1999-2000,BPXSY1,Systolic,Range of Values,True,74.0,242.0,NaN
4,1999-2000,BPXDI1,Diastolic,Range of Values,True,0.0,132.0,NaN


In [5]:
# map all non-range attributes to actual values according to codebook
# some categorical values also have "Range of Values" as its value for some reason
quantitative_attributes = codebook_df[codebook_df['description'] == 'Range of Values']['attribute_name'].unique()
qualitative_attributes = [attr for attr in \
                          codebook_df[codebook_df['description'] != 'Range of Values']['attribute_name'].unique() \
                          if attr not in quantitative_attributes]

for q_attribute in qualitative_attributes:
    df[q_attribute] = df[q_attribute].astype('object')
    print('Mapping:', q_attribute)
    for year_range in df['Year Range'].unique():
        col_year_range_codebook = codebook_df[(codebook_df['year_range'] == year_range) &
                                              (codebook_df['attribute_name'] == q_attribute)]
        
        # codebook for year_range + column doesnt exist
        if len(col_year_range_codebook) == 0:
            print(' - Unable to locate mapping for', year_range)

        # map all rows with corresponding year range and column to each value in codebook
        for index, row in col_year_range_codebook.iterrows():
            mask = (df['Year Range'] == year_range) & (df[q_attribute] == row['single_value'])
            if row['description'] != 'Range of Values':
                df.loc[mask, q_attribute] = row['description']

Mapping: Gender
Mapping: Vigorous-work
Mapping: Diabetes
Mapping: Coronary heart disease
Mapping: Blood related diabetes
Mapping: Blood related stroke
 - Unable to locate mapping for 2005-2006
 - Unable to locate mapping for 2007-2008
 - Unable to locate mapping for 2009-2010
 - Unable to locate mapping for 2011-2012
 - Unable to locate mapping for 2013-2014
 - Unable to locate mapping for 2015-2016
 - Unable to locate mapping for 2017-2018


From our EDA we discovered that "Blood related stroke" doesn't exist past 2005, therefore its mapping also doesn't exist. 

## Filtering

#### Coronary Heart Disease: Include only "Yes" or "No" values

In [6]:
df = df[df['Coronary heart disease'].isin(['Yes', 'No'])]
df['Coronary heart disease'].unique()

array(['No', 'Yes'], dtype=object)

#### Moderate-work: Drop nulls

In [7]:
df = df[~df['Moderate-work'].isna()]
(df['Moderate-work'].isna()).sum()

np.int64(0)

#### Vigorous-work: Drop nulls

In [8]:
df = df[~df['Vigorous-work'].isna()]
(df['Vigorous-work'].isna()).sum()

np.int64(0)

#### Diabetes: Drop nulls

In [9]:
df = df[~df['Diabetes'].isna()]
(df['Diabetes'].isna()).sum()

np.int64(0)

#### Blood related stroke, SEQN, Year Range: Drop Column

In [10]:
ignore_columns = ['Blood related stroke', 'SEQN', 'Year Range']
df = df[[col for col in df.columns if col not in ignore_columns]]
df.columns.unique()

Index(['Weight', 'Body mass index', 'Systolic', 'Diastolic', 'Gender', 'Age',
       'Diabetes', 'Glycohemoglobin', 'Cholesterol',
       'High-density lipoprotein (HDL)', 'Albumin',
       'Alanine aminotransferase (ALT)', 'Aspartate aminotransferase (AST)',
       'Alkaline phosphatase (ALP)', 'Gamma-glutamyl transferase (GGT)',
       'Glucose', 'Iron', 'Lactate dehydrogenase (LDH)', 'Phosphorus',
       'Bilirubin', 'Protein', 'Triglycerides', 'Uric acid', 'Creatinine',
       'White blood cells', 'Basophils', 'Red blood cells', 'Hemoglobin',
       'Red blood cell width', 'Platelet count', 'Mean volume of platelets',
       'Coronary heart disease', 'Blood related diabetes', 'Moderate-work',
       'Vigorous-work'],
      dtype='object')

## Interpolating Missing Values

#### Mean volume of platelets, Platelet count, Red blood cell width, Hemoglobin, Red blood cells, Basophils, White blood cells, Creatinine, Uric acid, Triglycerides, Protein, Bilirubin, Phosphorus, Lactate dehydrogenase (LDH), Iron, GLucose, Gamma-glutamyl transferase (GGT), Alkaline phosphatase (ALP), Aspartate aminotransferase (AST), Alanine aminotransferase (ALT), ALbumin, High-density liboprotein(HDL), Cholesterol. Glycohemoglobin, Diastolic, Systolic, Body mass index, Weight: Mean Imputation

In [11]:
columns_to_impute = [
    "Mean volume of platelets", 
    "Platelet count", 
    "Red blood cell width", 
    "Hemoglobin", 
    "Red blood cells", 
    "Basophils", 
    "White blood cells",
    "Creatinine", 
    "Uric acid", 
    "Triglycerides", 
    "Protein", 
    "Bilirubin", 
    "Phosphorus", 
    "Lactate dehydrogenase (LDH)", 
    "Iron", 
    "Glucose", 
    "Gamma-glutamyl transferase (GGT)", 
    "Alkaline phosphatase (ALP)", 
    "Aspartate aminotransferase (AST)", 
    "Alanine aminotransferase (ALT)", 
    "Albumin", 
    "High-density lipoprotein (HDL)", 
    "Cholesterol", 
    "Glycohemoglobin", 
    "Diastolic", 
    "Systolic", 
    "Body mass index", 
    "Weight", 
    "Age"
]

for column in columns_to_impute:
    mean_value = df[column].mean()
    num_null = df[column].isna().sum()
    print(f'"{column}": replacing {num_null} missing values with: {mean_value}')
    df.loc[df[column].isna(), column] = mean_value

"Mean volume of platelets": replacing 3988 missing values with: 8.187755785044942
"Platelet count": replacing 3988 missing values with: 249.19437033849687
"Red blood cell width": replacing 3987 missing values with: 13.226904596849376
"Hemoglobin": replacing 3987 missing values with: 14.098505964190949
"Red blood cells": replacing 3987 missing values with: 4.671089092343366
"Basophils": replacing 4082 missing values with: 0.04477933777373137
"White blood cells": replacing 3988 missing values with: 7.274411694396634
"Creatinine": replacing 4577 missing values with: 79.0302114298184
"Uric acid": replacing 4587 missing values with: 322.84494700846415
"Triglycerides": replacing 4600 missing values with: 1.7241213973799125
"Protein": replacing 4629 missing values with: 71.78599208564977
"Bilirubin": replacing 4602 missing values with: 11.361305012373235
"Phosphorus": replacing 4584 missing values with: 1.1984446357551652
"Lactate dehydrogenase (LDH)": replacing 4761 missing values with: 134.

In [12]:
print('\nNumber of remaining nulls:', df.isna().sum().sum())


Number of remaining nulls: 0


## Save resulting dataset

In [13]:
df.shape

(45820, 35)

In [14]:
df.to_parquet(resulting_filepath)